In [8]:
##
# File: sipp.ipynb
# Purpose: Data analysis for the 2020 Census SIPP (Survey of Income and Program Participation).
#	Can read the entire dataset csv (4GB) and extract relevant columns. Relevant columns (those relating to retirement
#	accounts and identifiers) are saved to a smaller csv for analysis.
#	Also runs calculations on relevant files and saves them for use in viz.

import pandas as pd
import numpy as np

# Snippets from census.gov data usage guide
rd_schema = pd.read_json('data/pu2021_schema.json')

rd_schema['dtype'] = ['Int64' if x == 'integer' \
			else 'object' if x == 'string' \
			else 'Float64' if x == 'float' \
			else 'ERROR' \
			for x in rd_schema['dtype']]

In [9]:
# USE THIS CELL TO EXTRACT COLUMNS FROM WHOLE DATASET -- DONT USE FOR CALCULATIONS
# I (Amy) am not committing the full dataset to Git. If you need more columns, let me know and I will extract them.

# Snippet from census.gov
df_data = pd.read_csv("data/pu2021.csv",\
		names=rd_schema['name'],\
		#dtype expects a dictionary of key:values
		dtype = dict([(i,v) for i,v in zip(rd_schema['name'], rd_schema['dtype'])]),\
		#files are pipe-delimited
		sep='|',\
		header=0,\
		#Add variables for analysis here. If you receive an out-of-memory error,
		#	either select less columns, or consider using the Dask module
		usecols = [
		#Common record-identification variables
		'SSUID','PNUM','MONTHCODE','ERESIDENCEID','ERELRPE','SPANEL','SWAVE',\
		#Common demographics variables, including age at time of interview (TAGE)
		#	and monthly age during the reference period (TAGE_EHC)
		'ESEX','TAGE','TAGE_EHC','ERACE','EORIGIN','EEDUC',\
		#Additional variables for analysis
		'EOWN_IRAKEO', 'EOWN_THR401','EOWN_SAV','EOWN_CD','EOWN_ST'
			]
		)
#preview the data		
print(df_data.head())
df_data.to_csv('data/sipp_data.csv') # Smaller csv that we can work with more easily

            SSUID  SPANEL  SWAVE  PNUM  ERELRPE  ESEX  EORIGIN  ERACE  EEDUC  \
0  00011428507021    2021      1   101        2     1        2      2     42   
1  00011428507021    2021      1   101        2     1        2      2     42   
2  00011428507021    2021      1   101        2     1        2      2     42   
3  00011428507021    2021      1   101        2     1        2      2     42   
4  00011428507021    2021      1   101        2     1        2      2     42   

   EOWN_SAV  EOWN_CD  EOWN_ST  EOWN_IRAKEO  EOWN_THR401  MONTHCODE  \
0         1        2        2            2            2          1   
1         1        2        2            2            2          2   
2         1        2        2            2            2          3   
3         1        2        2            2            2          4   
4         1        2        2            2            2          5   

  ERESIDENCEID  TAGE  TAGE_EHC  
0       100001    32        30  
1       100001    32        30  

In [34]:
# USE THIS CELL WHEN CALCULATING
data = pd.read_csv('data/sipp_data.csv')
unique_data = data.drop_duplicates(subset=["SSUID"])

#print(unique_data.head())

num_IRA = data[data['EOWN_IRAKEO'] == 1].count()['EOWN_IRAKEO']
num_401 = data[data['EOWN_THR401'] == 1].count()['EOWN_THR401']
num_SAV = data[data['EOWN_SAV'] == 1].count()['EOWN_SAV']
num_CD = data[data['EOWN_CD'] == 1].count()['EOWN_CD']
print(num_IRA, num_401, num_SAV, num_CD)
#print(num_IRA)

126156 164928 353364 28404


In [51]:
ages = unique_data.groupby('TAGE')['TAGE']

accounts_by_age = {}
for (i, _) in ages:
    #print(i)
    if not i in accounts_by_age:
        accounts_by_age[i] = {}
    aged_data = data[data['TAGE'] == i]
    accounts_by_age[i]['num_SAV'] = int(aged_data[aged_data['EOWN_SAV'] == 1].count()['EOWN_SAV'])
    accounts_by_age[i]['num_401'] = int(aged_data[aged_data['EOWN_THR401'] == 1].count()['EOWN_THR401'])
    accounts_by_age[i]['num_CD'] = int(aged_data[aged_data['EOWN_CD'] == 1].count()['EOWN_CD'])
    accounts_by_age[i]['num_IRA'] = int(aged_data[aged_data['EOWN_IRAKEO'] == 1].count()['EOWN_IRAKEO'])

{15: {'num_SAV': 1176, 'num_401': 24, 'num_CD': 48, 'num_IRA': 60}, 16: {'num_SAV': 2316, 'num_401': 0, 'num_CD': 72, 'num_IRA': 96}, 17: {'num_SAV': 2640, 'num_401': 24, 'num_CD': 60, 'num_IRA': 108}, 18: {'num_SAV': 2556, 'num_401': 0, 'num_CD': 96, 'num_IRA': 48}, 19: {'num_SAV': 2964, 'num_401': 108, 'num_CD': 72, 'num_IRA': 96}, 20: {'num_SAV': 4068, 'num_401': 444, 'num_CD': 72, 'num_IRA': 288}, 21: {'num_SAV': 4044, 'num_401': 624, 'num_CD': 84, 'num_IRA': 264}, 22: {'num_SAV': 3888, 'num_401': 552, 'num_CD': 72, 'num_IRA': 300}, 23: {'num_SAV': 4428, 'num_401': 1212, 'num_CD': 132, 'num_IRA': 456}, 24: {'num_SAV': 4392, 'num_401': 1608, 'num_CD': 108, 'num_IRA': 612}, 25: {'num_SAV': 4428, 'num_401': 2028, 'num_CD': 60, 'num_IRA': 612}, 26: {'num_SAV': 4248, 'num_401': 2340, 'num_CD': 96, 'num_IRA': 900}, 27: {'num_SAV': 4248, 'num_401': 2280, 'num_CD': 84, 'num_IRA': 972}, 28: {'num_SAV': 5112, 'num_401': 2772, 'num_CD': 96, 'num_IRA': 1080}, 29: {'num_SAV': 4644, 'num_401': 2

In [57]:
# Save to JSON for viz
import json

to_write = {
    "account_nums": {
        "num_IRA": int(num_IRA),
        "num_401k": int(num_401),
        "num_savings": int(num_SAV),
        "num_CD": int(num_CD)
    },
    "accounts_by_age": {}
}

for i in accounts_by_age:
    to_write['accounts_by_age'][str(i)] = accounts_by_age[i]

print(to_write)
    
json_object = json.dumps(to_write, indent=4)
with open("data/sipp.json", "w") as outfile:
    outfile.write(json_object)

{'account_nums': {'num_IRA': 126156, 'num_401k': 164928, 'num_savings': 353364, 'num_CD': 28404}, 'accounts_by_age': {'15': {'num_SAV': 1176, 'num_401': 24, 'num_CD': 48, 'num_IRA': 60}, '16': {'num_SAV': 2316, 'num_401': 0, 'num_CD': 72, 'num_IRA': 96}, '17': {'num_SAV': 2640, 'num_401': 24, 'num_CD': 60, 'num_IRA': 108}, '18': {'num_SAV': 2556, 'num_401': 0, 'num_CD': 96, 'num_IRA': 48}, '19': {'num_SAV': 2964, 'num_401': 108, 'num_CD': 72, 'num_IRA': 96}, '20': {'num_SAV': 4068, 'num_401': 444, 'num_CD': 72, 'num_IRA': 288}, '21': {'num_SAV': 4044, 'num_401': 624, 'num_CD': 84, 'num_IRA': 264}, '22': {'num_SAV': 3888, 'num_401': 552, 'num_CD': 72, 'num_IRA': 300}, '23': {'num_SAV': 4428, 'num_401': 1212, 'num_CD': 132, 'num_IRA': 456}, '24': {'num_SAV': 4392, 'num_401': 1608, 'num_CD': 108, 'num_IRA': 612}, '25': {'num_SAV': 4428, 'num_401': 2028, 'num_CD': 60, 'num_IRA': 612}, '26': {'num_SAV': 4248, 'num_401': 2340, 'num_CD': 96, 'num_IRA': 900}, '27': {'num_SAV': 4248, 'num_401':

TypeError: Object of type int64 is not JSON serializable